# アンドロイドは「青信号」の夢を見るか？

## 概要

- **背景:** 日本語の「青」は、物理的な緑色スペクトルの一部（信号機、野菜など）を包含する性質を持つ
- **RQ:** 言語データで学習されたVision-Language Models (VLMs) は、この言語的なねじれ（サピア・ウォーフ効果）を視覚認識において再現するか？すなわち、**日本語特化モデルは英語モデルに比べて、「緑色」を「青」と判定する確率が高いか？**
- **アプローチ:** CIELAB色空間上で制御された画像に対するモデルの応答曲線を計測

## 実験設定

### タスク定義とデータセット構築

- CIELAB色空間上で制御された、2種類の画像 (タスク) に対するモデルの応答曲線を計測
  1. 脱文脈化された色彩 (Decontextualized Color Patches)
  2. 文脈化された色彩 (Contextualized Objects)
- 各プロンプトテンプレートの埋め込みベクトルと画像埋め込みのコサイン類似度を計算し、Softmax関数で確率化
- 複数プロンプトの場合は平均を取る

#### タスク1: 脱文脈化された色彩 (Decontextualized Color Patches)

知覚的な均等性を保つため、RGB/HSVではなく **CIELAB ($L^*C^*h$) 色空間** を使用

- 明度 ($L^*$) と 彩度 ($C^*$) を固定し、色相角 ($h$) のみを変化させた単色パッチ。
- **Range:** $h \in [120^\circ, 240^\circ]$ (Green $\to$ Blue)
- **Step:** $1^\circ$ 刻み (計121枚)
- **Fixed Values:** $L^*=60, C^*=50$ (典型的な表示デバイスで再現可能な範囲)
- **Size:** $256 \times 256$ px

#### タスク2: 文脈化された色彩 (Contextualized Objects)

- 交通信号機 (Traffic Light) の画像における「点灯部分」のみをマスクし、上記と同様に色相回転 (Hue Shift) を適用する。
- これにより、「信号機」という物体認識が色彩判定に与えるバイアス (Top-down effect) を検証する。
- **枚数**: 1210 枚
  - $10$ 種類の信号画像それぞれに対して $1^\circ$ 刻み（計 $121$ 枚）の色相回転を適用

### モデル選定

#### 選定基準

- 公平な比較を期するため、なるべく同じベースモデル・パラメータサイズ、パッチサイズのモデルを使用

#### 選定モデル

| カテゴリ | モデルID (Hugging Face) |
| :--- | :--- |
| **English Native** | `openai/clip-vit-large-patch14` |
| **Japanese Native** | `llm-jp/llm-jp-clip-vit-large-patch14` |
| **Multilingual** | `sentence-transformers/clip-ViT-B-32-multilingual-v1` |

### プロンプトエンジニアリング

単一プロンプトのノイズを低減するため、**Prompt Ensembling** を採用する。複数のテンプレートの平均埋め込みベクトルを使用する。

- **英語 (En):**
  - `"a photo of a {color} color"`
  - `"a {color} patch"`
  - `"the color is {color}"`
  - `"this is {color}"`
  
- **日本語 (Ja):**
  - `"{色}の色の写真"`
  - `"{色}のパッチ"`
  - `"色は{色}です"`
  - `"これは{色}です"`

- **ターゲットラベル:**
  - En: `Blue` vs `Green`
  - Ja: `青` vs `緑`

### 評価指標

各色相角 $h_i$ における「青/Ao」の予測確率 $P(\text{Blue}|h_i)$ を算出し、以下の指標を計算する。

#### 1. Point of Subjective Equality (PSE: 主観的等価点)

「青」と「緑」の確率が等しくなる色相角。
$$
PSE = \{ h \mid P(\text{Blue}|h) = 0.5 \}
$$
※ 日本語モデルのPSEは、英語モデルよりも緑側（小さい角度）にシフトしていると仮定される。

#### 2. Cultural Color Shift ($\Delta PSE$)

英語モデルと日本語モデルの境界線のズレ。
$$
\Delta PSE = PSE_{Ja} - PSE_{En}
$$

#### 3. JND (Just Noticeable Difference) 近似 / Transition Width

確率が25%から75%に変化する色相の幅。境界の「曖昧さ」を表す。

## 実験結果

※ この表の数値を埋めていく

| 実験設定 | モデル | プロンプト | P(Blue\|h) |
|:--------:|:-------|:-----------|-----:|
| 1        | en     | en         |  |
| 1        | ja     | ja         |  |
| 1        | mul    | en         |  |
| 1        | mul    | ja         |  |
| 2        | en     | en         |  |
| 2        | ja     | ja         |  |
| 2        | mul    | en         |  |
| 2        | mul    | ja         |  |

## 参考

- 似た疑問があった: [How are boundaries between colors defined? Are they a cultural/linguistic/anthropological phenomenon stricto sensu, or are there biological/neurological bases behind color definition? : r/askscience](https://www.reddit.com/r/askscience/comments/vx4mql/how_are_boundaries_between_colors_defined_are/)

## その他補足やメモなど

### 初期アイデア

- **素朴な疑問:** 日本では緑色の信号を「青」と言います。また、「青菜」「青虫」も緑色です。**AI（CLIPなど）の視覚と言語の境界**において、「青」という言葉がカバーする「緑色の領域」はどこまで広いのか？
- **技術的アプローチ:**
    - **手法:** 色相環（カラーピッカー）で、「完全な青」から少しずつ「完全な緑」へ色を変化させた画像を用意する。
    - **判定:** 画像認識AI（CLIP）に画像を見せ、「これは『青』ですか？『緑』ですか？」という確率（Softmax）を出させる。
    - **比較:** 日本語モデルと英語モデル（Blue vs Green）で比較し、**日本語モデルだけが「緑色なのに青と言い張る領域（文化的な歪み）」**を可視化する。
    - **面白ポイント:** 言語が色の認識を歪めている様子を、数学的に可視化できます。「日本語AIは、このくらい緑でも『青だ』と言い張ります」という結果は知的好奇心をくすぐります。

In [ ]:
!nvidia-smi

Fri Dec 12 05:54:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   45C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 実験1用の画像を生成

